In [1]:
#set openai key in .env.local
import os
import pandas as pd
import openai
import numpy as np
#from openai.embeddings_utils import get_embedding

In [2]:
def estimate_price(n):
    num_tokens = 100 * n
    # per thousend tokens 0.0001 $
    price_per_thousend = 0.0001
    price = round(num_tokens/1000 * price_per_thousend, 5)

    response = input(f"Estimated price {price} $\nProceed? (y/n): ")
    if response.lower() == 'y':
        print("Price accepted...")
        # Continue with your notebook code here

    elif response.lower() == 'n':
        print("Price declined, interrupting...")
        raise KeyboardInterrupt  # Raises a KeyboardInterrupt to stop execution

    else:
        print("Invalid input. interrupting...")
        raise KeyboardInterrupt
        # Prompt the user again or take appropriate action


In [3]:
def set_environment_variables_from_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if line and not line.startswith('#'):  # Ignore empty lines and comments
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()
file_path = '../../.env.local'
set_environment_variables_from_file(file_path)
openai.api_key = os.getenv('OPENAI_KEY')

In [4]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [5]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'no_product_variants.pkl')
product_data = product_data.reset_index(drop=True)
product_data = product_data.sample(n=2000, random_state=2).reset_index(drop=True)
display(product_data)

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,3696208,4062862013129,Sneaker low Materialmix Leder/Textil grau,42.0,grau,1007.10.03,3692257,3692261,m,CN,...,Sneaker,Herrenschuhe,3 cm - 5 cm,42.0,8.0,7.0,0.0,Gummi-EVA,0.0,0.0
1,10310967,4065171998385,Slingpumps Rauleder Lila,38.5,Lila,21.521.13,6590677,315565,w,PT,...,Pumps,Damenschuhe,3 cm - 5 cm,38.5,5.5,9.0,0.0,TPU,0.0,0.0
2,3479727,4062862475613,Schnürstiefelette Rauleder gelb,42.0,gelb,52.076.30,6590678,315587,w,SK,...,Stiefeletten,Damenschuhe,bis 3 cm,42.0,8.0,14.0,14.0,TR,0.0,0.0
3,7593992,4064032904039,Plateau Sandale Materialmix Leder braun,35.0,braun,82.736.52,6590678,315568,w,VN,...,Sandalen,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,Gummi,1.0,1.0
4,8504015,4065171432780,Enkellaarsjes op plateau Suède grijs,44.0,grijs,94.780.13,6590677,315595,w,SK,...,Stiefeletten,Damenschuhe,5 cm - 8 cm,44.0,9.5,17.0,14.0,TR,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,7595163,4064032261774,Riemchensandale Glattleder braun,44.0,braun,82.773.51,6590678,315564,w,PT,...,Sandalen,Damenschuhe,3 cm - 5 cm,44.0,9.5,17.0,0.0,TR,0.0,0.0
1996,4770623,4062862808909,Plateau Sandale Rauleder,42.0,,64.614.17,6590677,315563,w,SK,...,Sandalen,Damenschuhe,5 cm - 8 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,1.0
1997,9218621,4065171587046,Plüschhausschuh Textil rosa,38.0,rosa,93.060.10,6590677,8487167,w,PT,...,Hausschuhe,Damenschuhe,bis 3 cm,38.0,5.0,6.0,0.0,EVA,0.0,0.0
1998,8666377,4065171660602,Lage sneaker Glad leer bruin,35.0,bruin,93.334.24,6590677,315567,w,SK,...,Sneaker,Damenschuhe,3 cm - 5 cm,35.0,2.5,3.0,0.0,TR,1.0,1.0


In [6]:
def combine_columns(row):
    return (
            "ID: " + str(row.product__id) + "; " +
            "product_number: " + str(row.productNumber) + "; " +
            "product_name: " + row.productName + "; " +
            # "size: " + str(row.productSizeEu) + "; " +
            "color: " + str(row.productColorName) + "; " +
            "article number: " + str(row.articleNumber) + "; " +
            "brand: " + str(row.brand__id) + "; " +
            "main category: " + str(row.mainCategory__id) + "; " +
            "gender: " + str(row.gender) + "; " +
            "line: " + str(row.line) + "; " +
            "price: " + str(row.b2cPrice) + "; " +
            "material: " + str(row.material) + "; " +
            "material PM: " + str(row.materialPm) + "; "
    )

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def get_embeddings(texts, model="text-embedding-ada-002"):
    return openai.Embedding.create(input=texts, model=model).data

def get_list_from_embeddings(embedding_result):
    list = []
    for entry in embedding_result:
        list.append(entry.embedding)
    return list

In [7]:
product_embeddings_path = "openai/product_embeddings.pkl"
recreate_embeddings = False
if not os.path.exists(basepath + product_embeddings_path) or recreate_embeddings:
    product_data_embeddings = product_data[['product__id']].copy()
    product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)
    # print(product_data_embeddings.combined.tolist())
    estimate_price(product_data_embeddings.shape[0])
    # embed each row separately
    # product_data_embeddings['embedding'] = product_data_embeddings.combined.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
    # batch embed
    product_data_embeddings['embedding'] = get_list_from_embeddings(get_embeddings(product_data_embeddings.combined.tolist()))
    product_data_embeddings.to_pickle(basepath + product_embeddings_path)
else:
    product_data_embeddings = pd.read_pickle(basepath + product_embeddings_path)
display(product_data_embeddings)

,product__id,combined,embedding
0,3696208,ID: 3696208; product_number: 4062862013129; pr...,"[0.013865606859326363, 0.0059647392481565475, ..."
1,10310967,ID: 10310967; product_number: 4065171998385; p...,"[0.008594418875873089, 0.018468571826815605, -..."
2,3479727,ID: 3479727; product_number: 4062862475613; pr...,"[0.0033649103716015816, -0.0010019299807026982..."
3,7593992,ID: 7593992; product_number: 4064032904039; pr...,"[0.008496461436152458, -0.00178595632314682, -..."
4,8504015,ID: 8504015; product_number: 4065171432780; pr...,"[0.01740363799035549, -0.01851450838148594, -0..."
...,...,...,...
1995,7595163,ID: 7595163; product_number: 4064032261774; pr...,"[0.006299823988229036, 0.009966087527573109, -..."
1996,4770623,ID: 4770623; product_number: 4062862808909; pr...,"[0.006249087397009134, -0.0014839477371424437,..."
1997,9218621,ID: 9218621; product_number: 4065171587046; pr...,"[0.006515574641525745, 0.005344858393073082, -..."
1998,8666377,ID: 8666377; product_number: 4065171660602; pr...,"[-0.0006240483489818871, -0.001225750078447163..."


In [13]:
product_data_embeddings.iloc[0].embedding

[0.013865606859326363,
 0.0059647392481565475,
 -0.02003992721438408,
 -0.020705262199044228,
 -0.005093148443847895,
 0.007844313979148865,
 -0.027864279225468636,
 -0.007704593241214752,
 -0.007119097281247377,
 -0.04149036481976509,
 0.02010646089911461,
 0.013639392331242561,
 -0.022701270878314972,
 -0.004747173748910427,
 0.0009655691683292389,
 0.008483036421239376,
 0.014118433929979801,
 0.014850304462015629,
 0.021437132731080055,
 -0.009347973391413689,
 0.006696608848869801,
 0.022554896771907806,
 -0.0015527283539995551,
 -0.004970061592757702,
 -0.0020043253898620605,
 -0.0075981393456459045,
 0.01779109053313732,
 -0.034065213054418564,
 0.0034431149251759052,
 -0.017604796215891838,
 -0.0014612446539103985,
 0.0020891556050628424,
 -0.03438457474112511,
 -0.008449769578874111,
 -0.008995345793664455,
 0.013825686648488045,
 -0.010465738363564014,
 -0.004288091789931059,
 0.011430475860834122,
 -0.002882569096982479,
 0.00572521798312664,
 0.011237528175115585,
 0.017764

In [8]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5):
    similarities = [cosine_similarity(query_embedding, neighbor) for neighbor in neighbors_embedding]
    indices = np.flip(np.argsort(similarities)[-n:])
    scores = [similarities[i] for i in indices]
    # Get the n neighbors with the highest similarity values
    n_neighbors = [neighbors_embedding[i] for i in indices]

    return indices, scores


indices, scores = get_n_nearest_neighbors(product_data_embeddings.embedding[0], product_data_embeddings.embedding)
display(indices, scores)

array([   0, 1450, 1027,   11,  762])

[1.0,
 0.9885263210629753,
 0.9866243723324932,
 0.9831100128526201,
 0.9819263753691818]

In [20]:
pd.set_option('display.max_colwidth', None)
for data in product_data_embeddings.iloc[indices].combined:
    print(data.replace('; ', ";\n"), "\n")

ID: 3696208;
product_number: 4062862013129;
product_name: Sneaker low Materialmix Leder/Textil grau;
color: grau;
article number: 1007.10.03;
brand: 3692257;
main category: 3692261;
gender: m;
line: H-W;
price: 120.0;
material: Materialmix Leder-Textil;
material PM: Suede/Velvet/Nylon;
 

ID: 7594042;
product_number: 4064032158098;
product_name: Sneaker low Materialmix Leder/Lederimitat Grau;
color: Grau;
article number: 86.986.40;
brand: 4997827;
main category: 315567;
gender: w;
line: F-S;
price: 130.0;
material: Materialmix Leder-Lederimitat;
material PM: Mesh/Loop/Velour/Ch;
 

ID: 3845441;
product_number: 4060666975391;
product_name: Sneaker low Materialmix Leder grau;
color: grau;
article number: 56.968.60;
brand: 4997827;
main category: 315567;
gender: w;
line: H-W;
price: 140.0;
material: Materialmix Leder;
material PM: Leopard Mini/Nubuk;
 

ID: 3696172;
product_number: 4062862012818;
product_name: Sneaker low Materialmix Leder/Textil schwarz;
color: schwarz;
article number: 1

In [10]:
product_data.iloc[indices]

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,3696208,4062862013129,Sneaker low Materialmix Leder/Textil grau,42.0,grau,1007.10.03,3692257,3692261,m,CN,...,Sneaker,Herrenschuhe,3 cm - 5 cm,42.0,8.0,7.0,0.0,Gummi-EVA,0.0,0.0
1450,7594042,4064032158098,Sneaker low Materialmix Leder/Lederimitat Grau,38.0,Grau,86.986.40,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,Gummi-EVA,0.0,0.0
1027,3845441,4060666975391,Sneaker low Materialmix Leder grau,37.5,grau,56.968.60,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,37.5,4.5,7.0,0.0,Gummi-EVA,0.0,1.0
11,3696172,4062862012818,Sneaker low Materialmix Leder/Textil schwarz,44.0,schwarz,1007.10.01,3692257,3692261,m,CN,...,Sneaker,Herrenschuhe,3 cm - 5 cm,44.0,9.5,10.0,0.0,Gummi-EVA,0.0,0.0
762,3695399,4062862138150,PG0496 Sneaker low Materialmix Leder/Textil grau,42.0,grau,0496.10.02,3692257,3692261,m,PT,...,Sneaker,Herrenschuhe,bis 3 cm,42.0,8.0,7.0,0.0,TR,0.0,0.0


In [11]:
def get_best_recommendations_multiple(query_embeddings, neighbors_embeddings, n = 5):
    all_results = {}
    for query_embedding in query_embeddings:
        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)
        result_dict = {index: score for index, score in zip(indices, scores)}
        for key, value in result_dict.items():
            if key in all_results:
                all_results[key] = max(all_results[key], value)
            else:
                all_results[key] = value
    return dict(sorted(all_results.items(), key=lambda item: item[1], reverse=True)[:n])


best_items = get_best_recommendations_multiple([product_data_embeddings.embedding[0], product_data_embeddings.embedding[1]], product_data_embeddings.embedding)

print(best_items)

{0: 1.0, 1: 1.0, 127: 0.9986555039769761, 1450: 0.9885263210629753, 1027: 0.9866243723324932}
